# Internet Usage Analysis

Data Sources: 
* [The World Bank, Internet usage](https://data.worldbank.org/indicator/IT.NET.USER.ZS?end=2022&start=1960&view=chart&year=2021)
* [The World Bank, populations](https://data.worldbank.org/indicator/SP.POP.TOTL?year=2021)

***

In [4]:
import pandas as pd
import numpy as np

## Data Exploration

In [5]:
data_file1 = "API_IT/API_IT.NET.USER.ZS_DS2_en_csv_v2_259708.csv"
data_file2 = "API_IT/Metadata_Country_API_IT.NET.USER.ZS_DS2_en_csv_v2_259708.csv"
data_file3 = "API_IT/Metadata_Indicator_API_IT.NET.USER.ZS_DS2_en_csv_v2_259708.csv"

The first data file contains the percentage of a country's population using the Internet from 1960 through 2023, though not every country listed has data for all of those years:

In [6]:
df1 = pd.read_csv(data_file1)
col_list = list(df1.columns)
col_list.pop()
col_list
df1 = df1[col_list]
df1.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,ABW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,83.780000,88.661227,93.542454,97.170000,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.237716,14.485849,15.753330,17.310987,20.063024,22.589591,24.988401,27.660654,NaN,NaN
2,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,7.000000,8.260000,11.000000,13.500000,16.800000,17.600000,18.400000,NaN,NaN,NaN
3,Africa Western and Central,AFW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,14.923441,18.109485,22.869750,27.081219,32.089337,35.276448,41.749819,46.990497,NaN,NaN
4,Angola,AGO,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,21.400000,22.000000,23.200000,26.000000,29.000000,32.129392,32.550147,32.602302,NaN,NaN


The second data file maps countries and regions to regions and income groups, and also includes a column with notes specific to that country or region:

In [7]:
df2 = pd.read_csv(data_file2)
col_list = list(df2.columns)
col_list.pop()
col_list
df2 = df2[col_list]
df2.head()

,Country Code,Region,IncomeGroup,SpecialNotes,TableName
0,ABW,Latin America & Caribbean,High income,NaN,Aruba
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan
3,AFW,NaN,NaN,"22 countries, stretching from the westernmost ...",Africa Western and Central
4,AGO,Sub-Saharan Africa,Lower middle income,The World Bank systematically assesses the app...,Angola


The third data file contains metadata about the other two data files:

In [8]:
df3 = pd.read_csv(data_file3)
# df3.head()
print("Indicator Name:", df3["INDICATOR_NAME"][0])
print("Indicator Explanation:", df3["SOURCE_NOTE"][0])
print("Data Source:", df3["SOURCE_ORGANIZATION"][0])

Indicator Name: Individuals using the Internet (% of population)
Indicator Explanation: Internet users are individuals who have used the Internet (from any location) in the last 3 months. The Internet can be used via a computer, mobile phone, personal digital assistant, games machine, digital TV etc.
Data Source: International Telecommunication Union (ITU) World Telecommunication/ICT Indicators Database


## Data Transformation

Let's remove rows from the second dataset that correspond to a region, so we're left with a dataset where each row corresponds to a single country:

In [9]:
subdf2 = df2[~df2["Region"].isna()]
assert df2.shape[0] > subdf2.shape[0]
print(subdf2.shape)

(217, 5)


In [10]:
subdf2["Region"].value_counts()

Europe & Central Asia         58
Sub-Saharan Africa            48
Latin America & Caribbean     42
East Asia & Pacific           37
Middle East & North Africa    21
South Asia                     8
North America                  3
Name: Region, dtype: int64

In [11]:
countries = list(subdf2["TableName"])
print(countries)

['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra', 'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The', 'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan', 'Botswana', 'Central African Republic', 'Canada', 'Switzerland', 'Channel Islands', 'Chile', 'China', "Côte d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica', 'Cuba', 'Curaçao', 'Cayman Islands', 'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt, Arab Rep.', 'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji', 'France', 'Faroe Islands', 'Micronesia, Fed. Sts.', 'Gabon', 'United Kingdom', 'Georgia', 'Ghana', 'Gibraltar', 'Guinea', 'Gambia, The', 'Guinea-Bi

Now let's join the second dataset to the first dataset, keeping only the rows included in the subset of the second dataset we just created.  This will give us a dataset of Internet usage by country that we can then group by region of the world and by income level.

In [12]:
df = subdf2[["Country Code", "Region", "IncomeGroup", "SpecialNotes"]].join(df1.set_index("Country Code"), on=["Country Code"], how="left")
assert df.shape[0] == subdf2.shape[0], "The total rows should be the same as the rows in the filtered, second dataset."
df.head()

,Country Code,Region,IncomeGroup,SpecialNotes,Country Name,Indicator Name,Indicator Code,1960,1961,1962,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,...,83.78,88.661227,93.542454,97.170000,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,...,7.00,8.260000,11.000000,13.500000,16.8,17.600000,18.400000,NaN,NaN,NaN
4,AGO,Sub-Saharan Africa,Lower middle income,The World Bank systematically assesses the app...,Angola,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,...,21.40,22.000000,23.200000,26.000000,29.0,32.129392,32.550147,32.602302,NaN,NaN
5,ALB,Europe & Central Asia,Upper middle income,NaN,Albania,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,...,54.30,56.900000,59.600000,62.400000,65.4,68.550391,72.237677,79.323718,82.613686,NaN
6,AND,Europe & Central Asia,High income,NaN,Andorra,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,...,86.10,87.900000,89.700000,91.567467,NaN,90.718665,93.205649,93.897522,NaN,NaN


Calculate average Internet usage by region:

In [13]:
df_region = df.groupby("Region").mean()
df_region

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Region,,,,,,,,,,,,,,,,,,,,,
East Asia & Pacific,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,42.786945,46.000058,49.619273,54.537359,63.301035,63.376842,67.940570,72.484991,86.849730,NaN
Europe & Central Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.643795,71.304608,74.249972,75.652162,79.132387,81.603273,83.935472,86.509749,88.586524,NaN
Latin America & Caribbean,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,49.297190,53.207043,57.443358,60.604695,61.343997,68.474006,72.115487,75.376399,79.352846,NaN
Middle East & North Africa,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,52.923769,57.867052,61.168901,67.203468,72.203980,76.574410,79.697989,85.255103,88.516890,NaN
North America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.640000,87.625937,91.568140,92.782087,91.569452,90.671591,91.460235,92.293612,NaN,NaN
South Asia,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,18.490000,21.770447,25.859431,30.141250,35.695163,41.137046,44.137153,53.392366,NaN,NaN
Sub-Saharan Africa,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,14.041692,16.519635,19.427751,21.961916,27.017946,29.608867,33.726764,39.325726,35.478121,NaN


The latest year that every region of the world has Internet usage data is 2021.

Calculate average Internet usage by income level:

In [14]:
df_income = df.groupby("IncomeGroup").mean()
df_income

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
IncomeGroup,,,,,,,,,,,,,,,,,,,,,
High income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,76.486695,78.524201,81.004833,82.399161,85.895660,86.983212,88.664762,90.318588,91.463700,NaN
Low income,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,6.766082,8.154947,10.289698,11.904790,14.165238,16.175444,19.270183,22.949434,NaN,NaN
Lower middle income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.139123,25.290162,29.535164,34.007579,41.208387,44.690254,50.007407,55.520681,62.088728,NaN
Upper middle income,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,44.067899,48.520204,52.568384,57.125577,63.733567,68.038498,72.243775,75.687120,82.021781,NaN


Export the data recording the percentage of country populations regularly using the Internet, with countries mapped to world regions:

In [15]:
df.to_csv("InternetUsageByCountryAndRegion.csv")

## From Usage to Users

Use The World Bank's population data to calculate the *number of people* using the Internet around the world:

In [16]:
data_file4 = "API_SP/API_SP.POP.TOTL_DS2_en_csv_v2_267401.csv"
df4 = pd.read_csv(data_file4)

# Remove the last column as doesn't contain any data 
col_list = list(df4.columns)
col_list.pop()
col_list
df4 = df4[col_list]

# Exclude rows for regions so that each row in the dataset corresponds to a single country, 
# as in the dataset created above
country_codes = df[["Country Code"]]
subdf4 = country_codes.join(df4.set_index("Country Code"), on="Country Code", how="left")
assert subdf4.shape[0] == df.shape[0], "The new dataset should have the same number of rows as the previously created dataset (one row per country)."

subdf4.head()

,Country Code,Country Name,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,Aruba,"Population, total",SP.POP.TOTL,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,...,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0,NaN
2,AFG,Afghanistan,"Population, total",SP.POP.TOTL,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,...,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,41128771.0,NaN
4,AGO,Angola,"Population, total",SP.POP.TOTL,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,...,27128337.0,28127721.0,29154746.0,30208628.0,31273533.0,32353588.0,33428486.0,34503774.0,35588987.0,NaN
5,ALB,Albania,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,2837849.0,2811666.0,2777689.0,NaN
6,AND,Andorra,"Population, total",SP.POP.TOTL,9443.0,10216.0,11014.0,11839.0,12690.0,13563.0,...,71621.0,71746.0,72540.0,73837.0,75013.0,76343.0,77700.0,79034.0,79824.0,NaN


In [18]:
print(subdf4[["2021"]].sum()[0])

7864818184.0


In [46]:
new_df4 = subdf4.drop(columns=["Indicator Name", "Indicator Code"])
new_df4 = new_df4.set_index(["Country Code", "Country Name"])
new_df = df.drop(columns=["Indicator Name", "Indicator Code"])
new_df = new_df.set_index(["Country Code", "Region", "IncomeGroup", "SpecialNotes", "Country Name"])
# Multiply the population of each country by the proportion of that population using the Internet to
# calculate the number of people using the Internet in each country each year
df_internet_users = (new_df4 / 100) * new_df
df_internet_users = df_internet_users.reset_index()
df_internet_users.head()

,Country Code,Country Name,Region,IncomeGroup,SpecialNotes,1960,1961,1962,1963,1964,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,Aruba,Latin America & Caribbean,High income,NaN,NaN,NaN,NaN,NaN,NaN,...,8.679105e+04,9.243554e+04,9.810171e+04,1.024551e+05,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,South Asia,Low income,The reporting period for national accounts dat...,NaN,NaN,NaN,NaN,NaN,...,2.290135e+06,2.788039e+06,3.809983e+06,4.811861e+06,6163379.712,6.647432e+06,7.170890e+06,NaN,NaN,NaN
2,AGO,Angola,Sub-Saharan Africa,Lower middle income,The World Bank systematically assesses the app...,NaN,NaN,NaN,NaN,NaN,...,5.805464e+06,6.188099e+06,6.763901e+06,7.854243e+06,9069324.570,1.039501e+07,1.088102e+07,1.124902e+07,NaN,NaN
3,ALB,Albania,Europe & Central Asia,Upper middle income,NaN,NaN,NaN,NaN,NaN,NaN,...,1.568783e+06,1.639120e+06,1.714156e+06,1.793037e+06,1874609.904,1.956559e+06,2.049996e+06,2.230318e+06,2.294751e+06,NaN
4,AND,Andorra,Europe & Central Asia,High income,NaN,NaN,NaN,NaN,NaN,NaN,...,6.166568e+04,6.306473e+04,6.506838e+04,6.761067e+04,NaN,6.925735e+04,7.242079e+04,7.421097e+04,NaN,NaN


In [47]:
print(df_internet_users[["2021"]].sum()[0])

4785781914.211266


Globally, there were about 4.8 billion people using the Internet in 2021.

In [52]:
pd.set_option('display.float_format', str)

In [53]:
df_internet_users_region = df_internet_users.groupby("Region").sum()
df_internet_users_region

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Region,,,,,,,,,,,,,,,,,,,,,
East Asia & Pacific,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1019094757.762313,1090108162.6078162,1174580396.1885865,1231937003.882194,1339962517.1470041,1460795924.2526648,1589297482.3179343,1671557665.0064359,1486826717.0548925,0.0
Europe & Central Asia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,619608669.0615959,631977932.8989975,656909482.9795109,680655958.8924981,711498924.7976077,733639336.575513,756977877.2980322,783030938.484939,634626575.2628726,0.0
Latin America & Caribbean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,300137394.2911279,337978392.51565534,361215756.73100877,381921785.41207784,386011593.688092,423460587.02197295,457086541.3994567,477085821.28883934,302512564.0671529,0.0
Middle East & North Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,173097567.1536917,192706323.10347357,214335674.44110703,247299495.5421981,257730926.8168697,293005996.57734954,321080086.152993,328957336.7334443,165965241.27437195,0.0
North America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,263358167.12600002,271321159.49608076,309350534.4353967,317690743.8889927,324326616.753182,328186791.9326734,335497905.060076,340136796.97716576,0.0,0.0
South Asia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,225051747.6048,252105212.40290102,291192492.4811752,327629977.2345,372298688.79517937,521102950.4865408,733732141.5955971,792902001.7376972,0.0,0.0
Sub-Saharan Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,126258285.90277715,155385509.3045671,193088894.99621886,226278447.98034006,258976940.3602418,305399857.03824323,359302640.4385686,392111353.98274386,9990831.17909609,0.0


In [54]:
df_internet_users_income = df_internet_users.groupby("IncomeGroup").sum()
df_internet_users_income

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
IncomeGroup,,,,,,,,,,,,,,,,,,,,,
High income,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,920532194.7275047,943455732.0142081,1001303803.8131874,1025296430.0274292,1046015931.2017314,1068639335.2547418,1084901972.759747,1096064510.203867,499952840.6531859,0.0
Low income,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36001340.52956229,47050212.32490116,62138443.609424405,73772324.78138447,75448048.5031,92401618.64824605,111088911.58455583,96527103.80449206,0.0,0.0
Lower middle income,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,530265919.35575813,604137096.3491288,702371410.8392547,799171206.3451618,892663757.4347212,1114940173.1634417,1408965430.2038157,1540807738.9794676,167288587.4782471,0.0
Upper middle income,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1222596977.2294807,1319232416.3612535,1416414695.5911374,1496346376.9508255,1636678471.2186239,1789610316.8185282,1948018359.7145395,2052382561.2234387,1932680500.706953,0.0


Export the data recording the count of people regularly using the Internet by country each year, with countries mapped to world regions:

In [55]:
df.to_csv("InternetUsersByCountryAndRegion.csv")